<a href="https://colab.research.google.com/github/Ahm1027/Payloads-Model/blob/Debug-Tests/Payload_Model_Ngrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import Sequential
import threading

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeClassifier

from nltk import ngrams
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
from google.colab import drive
import binascii

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.models import save_model, load_model

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def get_hexdump(executable, num_of_ngrams=4):
  executable = open(executable, 'rb')
  data = executable.read()
  executable.close()
  data2hex = bin2hex(data)
  output = ngrams(data2hex.split(" "), num_of_ngrams)
  return [ ''.join(grams) for grams in output]

def bin2hex(data):
  data2hex = ""
  for byte in data:
    hex = str(binascii.hexlify(bytes([byte]))).split("'")
    if (len(hex) >= 2):
      data2hex += hex[1]+" "
    else:
      data2hex += hex[0]+" "
  return data2hex

# def generate_ngrams(text, WordsToCombine):
#     words = text.split()
#     output = []  
#     for i in range(len(words)- WordsToCombine+1):
#       word = ''
#       for i in range (i, i+WordsToCombine):
#         word = word + words[i]
#       output.append(word)
#     return output

# def calculateTotalCounts(uniqueOutput, output):
#   Uniquecounts = []
#   for i in range(len(uniqueOutput)):
#     count = 0
#     for j in range(len(output)):
#       if output[j] == uniqueOutput[i]:
#         count = count+1
#     Uniquecounts.append(count)
#   return Uniquecounts

In [ ]:
def vectorize(data):
  vectorizer = CountVectorizer()
  vectorizer.fit(data)
  vectorized = vectorizer.transform(data)
  return vectorizer, vectorized

In [ ]:
def arrangeAndSort(data_vectorizer, data_vectorized):
  data_arranged = list(zip(data_vectorizer.get_feature_names(), data_vectorized.sum(0).getA1()))
  data_sorted = sorted(data_arranged, key=lambda item: item[1], reverse=True)
  return data_sorted

In [ ]:
def resolveMalwareFiles(FOLDER_PATHS, num_of_features=2000):
  stopCheck, prev=0, 0
  globalFeatures = np.array([])
  for files in os.listdir(FOLDER_PATHS):
    filepath = r"/content/drive/MyDrive/Payloads/"+files
    data = get_hexdump(filepath)
    data_vectorizer, data_vectorized = vectorize(data)
    data_sorted = arrangeAndSort(data_vectorizer, data_vectorized)[:num_of_features]
    if (len(globalFeatures) == 0):
      globalFeatures = np.array(data_sorted)[:, 0].astype(str).tolist()
    else:
      globalFeatures = np.append(globalFeatures, np.array(data_sorted)[:, 0].astype(str).tolist())
      indexes = np.unique(globalFeatures, return_index=True)[1]
      globalFeatures = [globalFeatures[index] for index in sorted(indexes)]
    if (stopCheck == prev+10):
      prev = stopCheck
      print("%f percent completed"%((stopCheck/1000)*100), end=", ")
      print("Number of Features: %d"%(len(globalFeatures)))
      print("Number of files processed: %d"%(stopCheck))
    stopCheck += 1
  return globalFeatures

globalFeatures = resolveMalwareFiles(r"/content/drive/MyDrive/Payloads")
print(len(globalFeatures))

1.000000 percent completed, Number of Features: 12824
Number of files processed: 10
2.000000 percent completed, Number of Features: 21665
Number of files processed: 20
3.000000 percent completed, Number of Features: 32188
Number of files processed: 30
4.000000 percent completed, Number of Features: 39380
Number of files processed: 40
5.000000 percent completed, Number of Features: 42516
Number of files processed: 50
6.000000 percent completed, Number of Features: 48005
Number of files processed: 60
7.000000 percent completed, Number of Features: 53519
Number of files processed: 70
8.000000 percent completed, Number of Features: 57798
Number of files processed: 80
9.000000 percent completed, Number of Features: 61965
Number of files processed: 90
10.000000 percent completed, Number of Features: 66964
Number of files processed: 100
11.000000 percent completed, Number of Features: 69128
Number of files processed: 110
12.000000 percent completed, Number of Features: 71534
Number of files p

In [ ]:
len(globalFeatures)

412090

In [ ]:
np.save('features_np.npy', globalFeatures)

In [ ]:
globalFeatures = np.load(r'/content/drive/MyDrive/features_np.npy')
len(globalFeatures)

412090

In [ ]:
def flatten_array(A):
    rt = []
    for i in A:
      if isinstance(i, np.ndarray):
        print('inside')
        rt.extend(flatten(i))
      else: rt.append(i)
    return np.array(rt)

globalFeatures = np.array(globalFeatures)
globalFeatures = np.sort(globalFeatures)
globalFeatures = flatten_array(globalFeatures)

In [ ]:
def normalizeFeatures(data):
  data_scaled = (data - data.min()) / (data.max() - data.min())   #MinMax Normalization
  return data_scaled

In [ ]:
def feature_labels():
  labels = []
  for filename in os.listdir(r"/content/drive/MyDrive/Payloads"):
    labels.append(r"/content/drive/MyDrive/Payloads/"+filename)
  return labels

featurelabels = feature_labels()
print(len(featurelabels))

1082


In [ ]:
# def find_first_no_jit(item, vec):
#   index = np.where(vec == item)
#   if index:
#     return index
#   return -1
Merged_Vocab = []
def binarySearch(array, item):
  lowPoint = 0
  highpoint = len(array) - 1
  while (lowPoint <= highpoint):
      midPoint = int((lowPoint + highpoint) / 2)
      if item == array[midPoint]:
          return midPoint
      elif item < array[midPoint]:
          highpoint = midPoint - 1
      else:
          lowPoint = midPoint + 1
  return -1

def vocab_parallel_process(num_of_threads = 4):
  thread_load = int(len(featurelabels)/num_of_threads)
  threads = []
  label_count_start, label_count_stop= 0, thread_load
  for x in range(num_of_threads):
    x = threading.Thread(target=mergeWithGlobalVocab, args=(label_count_start, label_count_stop, featurelabels, ))
    label_count_start = label_count_stop+1
    label_count_stop += thread_load
    x.start()
  for thread in threads:
    thread.join()

def mergeWithGlobalVocab(startLabel, stopLabel, featurelabels):
  stopCheck, prev=0, 0
  for i in range(startLabel, stopLabel+1):
    data = get_hexdump(featurelabels[i])
    data_vectorizer, data_vectorized = vectorize(data)
    data_sorted = arrangeAndSort(data_vectorizer, data_vectorized)[:2000]
    features = [0]*globalFeatures.size
    features = np.array(features)
    for x in data_sorted:
      id = binarySearch(globalFeatures, x[0])
      if (id != -1):
        features[id] = x[1]
    Merged_Vocab.append(features)

  # for files in os.listdir(r"/content/drive/MyDrive/Payloads"):
  #   filepath = r"/content/drive/MyDrive/Payloads/"+files
  #   data = get_hexdump(filepath)
  #   data_vectorizer, data_vectorized = vectorize(data)
  #   data_sorted = arrangeAndSort(data_vectorizer, data_vectorized)[:2000]
  #   features = [0]*globalFeatures.size
  #   features = np.array(features)
    
  #   global_Merged.append(features)
  #   if (stopCheck == prev+10):
  #     prev = stopCheck
  #     print("%f percent completed"%((stopCheck/1080)*100), end=", ")
  #     print("Length of Merged Features: %d"%(len(global_Merged)), end=", ")
  #     print("Number of files processed: %d"%(stopCheck))
  #   stopCheck += 1
  # return np.array(global_Merged)

merged_features = vocab_parallel_process()

In [ ]:
Merged_Vocab = []
def binarySearch(array, item):
  lowPoint = 0
  highpoint = len(array) - 1
  while (lowPoint <= highpoint):
      midPoint = int((lowPoint + highpoint) / 2)
      if item == array[midPoint]:
          return midPoint
      elif item < array[midPoint]:
          highpoint = midPoint - 1
      else:
          lowPoint = midPoint + 1

  return -1

def mergeWithGlobalVocab(startLabel, stopLabel, featurelabels):
  stopCheck, prev=0, 0
  for i in range(startLabel, stopLabel+1):
    data = get_hexdump(featurelabels[i])
    data_vectorizer, data_vectorized = vectorize(data)
    data_sorted = arrangeAndSort(data_vectorizer, data_vectorized)[:2000]
    features = [0]*globalFeatures.size
    features = np.array(features)
    for x in data_sorted:
      id = binarySearch(globalFeatures, x[0])
      if (id != -1):
        features[id] = x[1]
    Merged_Vocab.append(features)

mergeWithGlobalVocab(0, 100, featurelabels)


In [ ]:
np.save('features_merged.npy', np.array(Merged_Vocab))

In [ ]:
compressed_SVD = TruncatedSVD(30)
compressed_data = compressed_SVD.fit_transform(Merged_Vocab)
pd.DataFrame(compressed_SVD.components_)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,412050,412051,412052,412053,412054,412055,412056,412057,412058,412059,412060,412061,412062,412063,412064,412065,412066,412067,412068,412069,412070,412071,412072,412073,412074,412075,412076,412077,412078,412079,412080,412081,412082,412083,412084,412085,412086,412087,412088,412089
0,5.284582e-14,0.000012,6.676304e-08,2.799553e-08,3.684871e-08,4.063389e-08,1.445506e-08,2.640837e-13,3.822548e-08,4.651989e-08,3.630898e-08,2.330843e-08,4.708264e-08,1.818123e-08,9.848289e-10,1.478869e-12,5.269418e-08,4.150040e-08,4.547934e-11,1.881361e-10,1.056335e-13,1.959044e-08,1.056335e-13,1.056335e-13,1.056335e-13,2.097746e-08,2.112669e-13,3.169004e-13,2.640837e-13,2.640837e-13,2.112669e-13,2.640837e-13,4.225339e-13,4.205832e-08,3.697171e-13,3.285022e-11,2.640837e-13,1.678217e-08,3.169004e-13,3.697171e-13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.123279e-08,0.0,0.0,2.190957e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.584948e-08,0.0,3.589441e-08,0.000002
1,2.205443e-11,0.000213,1.592417e-06,1.503900e-07,1.945911e-07,7.744131e-07,7.558771e-08,1.102722e-10,1.955600e-07,7.030980e-07,7.109827e-07,1.193310e-07,2.408912e-07,9.335476e-08,1.251921e-06,6.175244e-10,2.740643e-07,2.154595e-07,1.193520e-08,4.905471e-08,4.410889e-11,1.049824e-07,4.410889e-11,4.410889e-11,4.410889e-11,1.073472e-07,8.821778e-11,1.323267e-10,1.102722e-10,1.102722e-10,8.821778e-11,1.102722e-10,1.764356e-10,9.281528e-07,1.543811e-10,8.585910e-09,1.102722e-10,8.596156e-08,1.323267e-10,1.543811e-10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.596967e-07,0.0,0.0,1.120260e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.104011e-08,0.0,1.835320e-07,0.000034
2,4.714738e-07,0.936155,5.928911e-03,1.817900e-03,2.327199e-03,3.735930e-03,9.014816e-04,2.357369e-06,2.290893e-03,3.439801e-03,2.874116e-03,1.398749e-03,2.822106e-03,1.096936e-03,1.731479e-03,1.320127e-05,3.243056e-03,2.547800e-03,2.255218e-04,9.247893e-04,9.429476e-07,1.276372e-03,9.429476e-07,9.429476e-07,9.429476e-07,1.257790e-03,1.885895e-06,2.828843e-06,2.357369e-06,2.357369e-06,1.885895e-06,2.357369e-06,3.771790e-06,3.510605e-03,3.300317e-06,1.620014e-04,2.357369e-06,1.008023e-03,2.828843e-06,3.300317e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.869901e-03,0.0,0.0,1.311722e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.489050e-04,0.0,2.148991e-03,0.148087
3,-3.462467e-09,-0.003235,-2.116909e-05,-4.459445e-06,-5.834239e-06,-1.027450e-05,-2.518370e-06,-1.731233e-08,-5.973580e-06,-1.173877e-05,-1.043918e-05,-3.656108e-06,-7.360615e-06,-2.895183e-06,-1.157901e-05,-9.694907e-08,-8.090944e-06,-6.458199e-06,3.065320e-07,1.416268e-06,-6.924933e-09,-3.815976e-06,-6.924933e-09,-6.924933e-09,-6.924933e-09,-3.282533e-06,-1.384987e-08,-2.077480e-08,-1.731233e-08,-1.731233e-08,-1.384987e-08,-1.731233e-08,-2.769973e-08,-1.296990e-05,-2.423727e-08,2.377265e-07,-1.731233e-08,-2.639184e-06,-2.077480e-08,-2.423727e-08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.866706e-06,0.0,0.0,-3.413958e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.469672e-06,0.0,-5.593080e-06,-0.000521
4,-9.892325e-09,-0.003410,-3.052847e-05,1.812481e-06,2.656964e-06,-1.171034e-05,5.030575e-07,-4.946162e-08,3.296050e-06,-1.049880e-05,-1.245500e-05,1.970700e-06,4.051445e-06,1.413096e-06,-3.723607e-05,-2.769851e-07,4.663398e-06,3.476600e-06,1.614776e-07,1.059272e-06,-1.978465e-08,-4.726491e-07,-1.978465e-08,-1.978465e-08,-1.978465e-08,1.796382e-06,-3.956930e-08,-5.935395e-08,-4.946162e-08,-4.946162e-08,-3.956930e-08,-4.946162e-08,-7.913860e-08,-1.746416e-05,-6.924627e-08,1.597054e-07,-4.946162e-08,1.399515e-06,-5.935395e-08,-6.924627e-08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.733528e-06,0.0,0.0,1.917549e-06,0.0,0.0,0.0,0.0,0.0,0.0,0

In [ ]:
compressed_data.shape

(101, 30)

In [ ]:
payloads = np.vstack((globalFeatures, Merged_Vocab))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,412050,412051,412052,412053,412054,412055,412056,412057,412058,412059,412060,412061,412062,412063,412064,412065,412066,412067,412068,412069,412070,412071,412072,412073,412074,412075,412076,412077,412078,412079,412080,412081,412082,412083,412084,412085,412086,412087,412088,412089
/content/drive/MyDrive/Payloads/VirusShare_0aa6f0480ad5ca4df77d53b5f9990005,000000,00000000,00000001,00000002,00000003,00000004,00000005,00000006,00000007,00000008,00000009,0000000a,0000000b,0000000c,0000000d,0000000e,0000000f,00000010,00000011,00000012,00000013,00000014,00000015,00000016,00000017,00000018,00000019,0000001a,0000001b,0000001c,0000001d,0000001e,0000001f,00000020,00000021,00000022,00000023,00000024,00000025,00000026,...,ffffffd8,ffffffd9,ffffffda,ffffffdb,ffffffdc,ffffffdd,ffffffde,ffffffdf,ffffffe0,ffffffe1,ffffffe2,ffffffe3,ffffffe4,ffffffe5,ffffffe6,ffffffe7,ffffffe8,ffffffe9,ffffffea,ffffffeb,ffffffec,ffffffed,ffffffee,ffffffef,fffffff0,fffffff1,fffffff2,fffffff3,fffffff4,fffffff5,fffffff6,fffffff7,fffffff8,fffffff9,fffffffa,fffffffb,fffffffc,fffffffd,fffffffe,ffffffff
/content/drive/MyDrive/Payloads/VirusShare_0d6446c8bd36f446e243770626340b88,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
/content/drive/MyDrive/Payloads/VirusShare_00d775ceaad26694d43aeef105a437a7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
/content/drive/MyDrive/Payloads/VirusShare_0d6a237e1b434f02539cb973a04566e1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
/content/drive/MyDrive/Payloads/VirusShare_0b37d701517a9aa6387cb4271fdaeed3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
pd.DataFrame(Merged_Vocab)

In [ ]:
df.to_csv('features.csv')

In [ ]:
dt = DecisionTreeClassifier() 
clf = AdaBoostClassifier(n_estimators=100, base_estimator=dt,learning_rate=1)

In [ ]:
# training the model
clf.fit(Merged_Vocab, featurelabels[:101])

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=None,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                       

In [ ]:
Merged_Vocab2 = []

mergeWithGlobalVocab(101, 201, featurelabels)

In [ ]:
predictions = clf.predict(Merged_Vocab[150:])

In [ ]:
predictions

array(['/content/drive/MyDrive/Payloads/VirusShare_0b0f88fcce86e26789fa7b3009e9ebc2',
       '/content/drive/MyDrive/Payloads/VirusShare_0b5c93271ac6178dd40f8f2685a3a5d2',
       '/content/drive/MyDrive/Payloads/VirusShare_0b193211cd35fbd5c30b8f69a0fd47a3',
       '/content/drive/MyDrive/Payloads/VirusShare_0a91e3e2f16f97bbf783dc85d79518fe',
       '/content/drive/MyDrive/Payloads/VirusShare_0a575d2497de7f8377b3d9b80477b239',
       '/content/drive/MyDrive/Payloads/VirusShare_0b0f88fcce86e26789fa7b3009e9ebc2',
       '/content/drive/MyDrive/Payloads/VirusShare_0a3bb2cf77f2bb0e4fb437bf9778dc6f',
       '/content/drive/MyDrive/Payloads/VirusShare_0dad0be4d31340560be6d81688bef697',
       '/content/drive/MyDrive/Payloads/VirusShare_0d156396f1f3d023a339ea463d739a6d',
       '/content/drive/MyDrive/Payloads/VirusShare_0b0f88fcce86e26789fa7b3009e9ebc2',
       '/content/drive/MyDrive/Payloads/VirusShare_0c351c569903737b790aa5a25138cf84',
       '/content/drive/MyDrive/Payloads/VirusShare_0b0

In [ ]:
featurelabels

['/content/drive/MyDrive/Payloads/VirusShare_0aa6f0480ad5ca4df77d53b5f9990005',
 '/content/drive/MyDrive/Payloads/VirusShare_0d6446c8bd36f446e243770626340b88',
 '/content/drive/MyDrive/Payloads/VirusShare_00d775ceaad26694d43aeef105a437a7',
 '/content/drive/MyDrive/Payloads/VirusShare_0d6a237e1b434f02539cb973a04566e1',
 '/content/drive/MyDrive/Payloads/VirusShare_0b37d701517a9aa6387cb4271fdaeed3',
 '/content/drive/MyDrive/Payloads/VirusShare_0baa2b3ebb1f7d7b5d31b7fdf635b6ad',
 '/content/drive/MyDrive/Payloads/VirusShare_0d613a04dffaa9dbca574e142113ad62',
 '/content/drive/MyDrive/Payloads/VirusShare_0b8fab278c8a72f545fd88b6cfad065d',
 '/content/drive/MyDrive/Payloads/VirusShare_0a0af7482f256e5bc4ca2a67681268ae',
 '/content/drive/MyDrive/Payloads/VirusShare_0a575d2497de7f8377b3d9b80477b239',
 '/content/drive/MyDrive/Payloads/VirusShare_0a8fc685d43fde752598dda1b0a29842',
 '/content/drive/MyDrive/Payloads/VirusShare_0bc27d042cad9514309a97e26cc28ad1',
 '/content/drive/MyDrive/Payloads/VirusS